In [1]:
from data_gen import *
from models import *

In [3]:
# data generator
dg = DataGen(data="data/sig/wol-train.txt")

# length of a word
n_input_length = len(char2int)
n_steps_in = dg.max_root_len
n_steps_out = dg.max_output_len


5


In [4]:
print("Total train data: ", len(dg.words))
batch_size = 128
# number of batches to train
n_batches = int(len(dg.words) / batch_size) 

# python generator to generate training data at each request
# E.x word_matrix, feature = next(gen)
gen = dg.cnn_gen_data_multi_word(batch_size=batch_size, n_batches=n_batches)

Total train data:  14780


In [7]:
# infenc - inference encoder model
# infdec - inference decoder model
# train - training model that combines both
# n_input_length - the length of the input and the output
# word_feat_len - the length of the word feature vector
# n_units - size of the hidden memory in the RNN
model, encoder, decoder = conv_multi_model(n_input_length, n_input_length, dg.word_feat_len + 3, 512)
model.compile(optimizer='adam', loss='categorical_crossentropy')


In [8]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
root_word_input (InputLayer)    [(None, 15, 28, 1)]  0                                            
__________________________________________________________________________________________________
cnn (Conv2D)                    (None, 15, 28, 20)   520         root_word_input[0][0]            
__________________________________________________________________________________________________
pooling (MaxPooling2D)          (None, 5, 9, 20)     0           cnn[0][0]                        
__________________________________________________________________________________________________
flatten (Flatten)               (None, 900)          0           pooling[0][0]                    
____________________________________________________________________________________________

In [5]:
# model train given the data generator, how many batches and number of epochs
history = model.fit_generator(gen, steps_per_epoch=n_batches, epochs = 1)

W0731 22:21:50.720466  9832 deprecation.py:323] From C:\Users\amany\.conda\envs\tf2\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


115/115 [==============================] - 10s 83ms/step - loss: 1.5702


In [10]:
dg2 = DataGen(data="data/wolaytta-test.txt")
test_n_batches, test_batch_size =  int(len(dg2.words) / batch_size), batch_size  
print(test_n_batches * test_batch_size)
# test_n_batches, test_batch_size = 30, 10 

# data generator for test data
test_gen = dg2.cnn_gen_data_multi_word(batch_size=test_batch_size, n_batches=test_n_batches, trainset=False)

6
4352


In [11]:
def predict(infenc, infdec, inputs, n_steps, cardinality):
    # encode
    state = infenc.predict(inputs)
    # start of sequence input
    start = [0.0 for _ in range(cardinality)]
#     start[0] = 1
    target_seq = np.array(start).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h= infdec.predict([target_seq, state])
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = h
        # update target sequence
        target_seq = yhat
    return np.array(output)

In [23]:
# shows sample examples and calculates accuracy

total, correct = 0, 0
in_word = 0
sims = []
for b in range(1):
    # get data from test data generator
    [X1, X2, X3], y = next(test_gen)
    for j in range(test_batch_size):
        word_features = X3[j].reshape((1, X3.shape[1])) 
        root_word_matrix = X1[j].reshape((1, X1.shape[1], X1.shape[2], 1))
#         word_index = X4[j].reshape((1, X4.shape[1]))
        # predicts the target word given root word and features
        target = predict(encoder, decoder, [root_word_matrix, word_features], n_steps_out, n_input_length)
#         root = ''.join(dg.one_hot_decode(X1[j]))#.replace('&', ' ')
#         word = ''.join(dg.one_hot_decode(y[j]))#.replace('&', ' ')
#         targetS = ''.join(dg.one_hot_decode(target))#.replace('&', ' ')
#         sims.append(dg.word_sim(word, targetS))
        
        # checks if the predicted and the real words are equal
        if ''.join(dg.one_hot_decode(y[j])).strip() == ''.join(dg.one_hot_decode(target)).strip():
            correct += 1
#         else:
#             print(root, word.split('&')[0], '\t\t', targetS.split('&')[0])
#         if root.strip() in targetS.strip():
#             in_word += 1
#     print(b, root, word, targetS)
    total += test_batch_size
    

print('Exact Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

Exact Accuracy: 0.00%
